# Lesson 2 : Finding Gaussian Process hyperparameters

Below some packages to import that will be used for this lesson

Cell bellow is here for avoiding scrolling when plot is create within ipython notebook

In [21]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines){
    return false;
}

<IPython.core.display.Javascript object>

In [22]:
# Classical package for manipulating
# array, for plotting and interactiv plots.
import pylab as plt
from matplotlib import gridspec
import numpy as np
import ipywidgets as widgets
from ipywidgets import interact
import itertools

# Gaussian processes from scikit-learn is used for this lesson.
# Other packages exist (e.g. george) but for the courses I guess
# it would be the best because a lot of people in desc are already
# using scikit-learn. The only suggestion would be about how to fit
# hyperparameters in a more efficient way, but it will be done with
# an other packages also broadly used within DESC, which is TreeCorr.
from sklearn import gaussian_process as skl_gp
from sklearn.gaussian_process.kernels import Kernel

# Special implemetation of anisotropic squarred exponential kernel
# in scikit-learn. Not implemented in scikit-learn originally.
import sys
sys.path.append('/home/nbuser/project')
from kernel import AnisotropicRBF

# treecorr is a package to compute 2-point correlation function.
# it will be use as an alternative way of Maximum Likelihood described 
# in Rasmussen & Williams 2006 to estimate hyperparameters.
try:
    import treecorr
except:
    !pip install treecorr
    import treecorr
    
try:
    import emcee
except:    
    !pip install emcee
    import emcee

try:
    import corner
except:    
    !pip install corner
    import corner

# Some import trickery to get all subclasses of sklearn.gaussian_process.kernels.Kernel
# into the local namespace without doing "from sklearn.gaussian_process.kernels import *"
# and without importing them all manually. Originally developped by Josh Meyers within Piff.
# Example:
# kernel = eval_kernel("RBF(1)") instead of
# kernel = sklearn.gaussian_process.kernels.RBF(1)
def eval_kernel(kernel):
    def recurse_subclasses(cls):
        out = []
        for c in cls.__subclasses__():
            out.append(c)
            out.extend(recurse_subclasses(c))
        return out
    clses = recurse_subclasses(Kernel)
    for cls in clses:
        module = __import__(cls.__module__, globals(), locals(), cls)
        execstr = "{0} = module.{0}".format(cls.__name__)
        exec(execstr, globals(), locals())

    from numpy import array

    try:
        k = eval(kernel)
    except (KeyboardInterrupt, SystemExit):
        raise
    except Exception as e:  # pragma: no cover
        raise RuntimeError("Failed to evaluate kernel string {0!r}.  "
                               "Original exception: {1}".format(kernel, e))

    if type(k.theta) is property:
        raise TypeError("String provided was not initialized properly")
    return k

## Exercice 5) Maximum Likelihood search of best hyperparameters / kernel (example in 1D):

In [23]:
##########################################################################################
# EXERCICE 5: Maximum Likelihood search of best hyperparameters / kernel (example in 1D) #
##########################################################################################

def log_likelihood(param, kernel_type="RBF"):
    if param[1] <=0:
        return -np.inf
    else:
        Kernel = "%f * %s(%f)"%((param[0]**2, kernel_type, param[1]))
        Kernel = eval_kernel(Kernel)
    
        gp = skl_gp.GaussianProcessRegressor(kernel=Kernel, alpha=y_err,
                                             optimizer=None,
                                             normalize_y=None)
        gp.fit(x,y)
        log_L = gp.log_marginal_likelihood()
        return log_L


def mcmc_hyperparameters_search(run_mcmc=False):
    if run_mcmc:
        p0 = [1., 0.5]
        np.random.seed(42)
        ndim, nwalkers = len(p0), 100
        pos = [p0 + 1e-4*np.random.randn(ndim) for i in range(nwalkers)]

        sampler = emcee.EnsembleSampler(nwalkers, ndim, log_likelihood)
        sampler.run_mcmc(pos, 600)
        LABEL = ['$\sigma$','$l$']
        for j in range(ndim):
            plt.figure()
            for i in range(nwalkers):
                plt.plot(sampler.chain[i,:,j],'k', alpha=0.1)
            plt.ylabel(LABEL[j], fontsize=20)

        samples = sampler.chain[:, 60:, :].reshape((-1, ndim))
    
        fig = corner.corner(samples, labels=LABEL,
                            levels=(0.68, 0.95))
        return samples

data = np.loadtxt('data/data_1d_grf.txt')
x = data[:,0].reshape((len(data[:,0]),1))
y = data[:,1]
y_err = data[:,2]


def gp_regression(x, new_x, y, kernel, y_err=None):
    
    if y_err is None:
        y_err =np.ones_like(y) *1e-10
    
    gp = skl_gp.GaussianProcessRegressor(kernel=kernel, alpha=y_err,
                                         optimizer=None,
                                         normalize_y=None)
    gp.fit(x,y)
    y_predict, y_std = gp.predict(new_x, return_std=True)
    return y_predict, y_std

@interact(sigma = widgets.FloatSlider(value=1.2, min=0.75, max=2.5, step=0.01, description='$\sigma$:',
          disabled=False,
          continuous_update=False,
          orientation='horizontal',
          readout=True,
          readout_format='.2f'), 
          l = widgets.FloatSlider(value=0.6, min=0.4, max=1.5, step=0.01, description='$l$:',
          disabled=False,
          continuous_update=False,
          orientation='horizontal',
          readout=True,
          readout_format='.2f'),
          kernel = widgets.Dropdown(options=['RBF', 'Matern'],
                                  value='RBF',
                                  description='Kernel:',
                                  disabled=False,))
def plot_samples(sigma, l, kernel):
    
    new_x = np.linspace(-24,24, 400).reshape((400,1))
    Kernel = "%f * %s(%f)"%((sigma**2, kernel, l))
    Kernel = eval_kernel(Kernel)
    y_pred, y_std = gp_regression(x, new_x, y, Kernel, y_err=y_err)

    gs = gridspec.GridSpec(1, 2, width_ratios=[1.5, 1])
    plt.figure(figsize=(20,8))
    plt.subplot(gs[0])
    
    # Data
    plt.scatter(x, y, c='b', label = 'data')
    plt.errorbar(x, y, linestyle='', yerr=y_err, ecolor='b', 
                 alpha=0.7,marker='.',zorder=0)
    
    # GP prediction
    plt.plot(new_x, y_pred, 'r', lw =3, label = 'GP prediction')
    plt.fill_between(new_x.T[0], y_pred-y_std, y_pred+y_std, color='r', alpha=0.3)
    
    plt.plot(new_x, np.zeros_like(new_x),'k--')
    plt.xlim(-24,24)
    plt.ylim(-3.,3.)
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    plt.xlabel('X', fontsize=20)
    plt.ylabel('Y', fontsize=20)
    plt.legend(fontsize=18)
    
    plt.subplot(gs[1])
    
    distance = np.linspace(0, 2, 40)
    coord = np.array([distance, np.zeros_like(distance)]).T
    pcf = Kernel.__call__(coord, Y=np.zeros_like(coord))[:,0]
    
    plt.plot(distance, pcf, 'k', lw=3)
    
    plt.ylim(0, 2.5**2)
    plt.xlim(0, 2)
    plt.ylabel('$\\xi(|x_i-x_j|)$', fontsize=20)
    plt.xlabel('$|x_i-x_j|$', fontsize=20)
    plt.title('Used correlation function (%s)'%(kernel), fontsize=16)
    
    samples = np.loadtxt('data/data_1d_grf_mcmc_likelihood_sampling_%s.txt'%(kernel))
    fig = corner.corner(samples, labels=['$\sigma$','$l$'],
                        truths=[sigma, l, ], levels=(0.68, 0.95))
    fig.suptitle('Kernel type: ' + kernel + ', $\log$ likelihood = %.2f'%(log_likelihood([sigma, l], kernel_type=kernel)))

interactive(children=(FloatSlider(value=1.2, continuous_update=False, description='$\\sigma$:', max=2.5, min=0…

## Exercice 6) 2-point correlation function search of best hyperparameters / kernel (example in 1D) :

In [24]:
##########################################################################################
# EXERCICE 6: Maximum Likelihood search of best hyperparameters / kernel (example in 1D) #
##########################################################################################

data = np.loadtxt('data/data_1d_grf_4000_points.txt')
x = data[:,0].reshape((len(data[:,0]),1))
y = data[:,1]
y_err = data[:,2]

np.random.seed(42)
Filter = np.random.choice([True, False, False, False, False], size=len(y))

cat = treecorr.Catalog(x=x[:,0], y=np.zeros_like(x[:,0]), k=(y-np.mean(y)), w=1./y_err**2)
kk = treecorr.KKCorrelation(min_sep=0.05, max_sep=1.5, nbins=15.)
kk.process(cat)
delta_distance = kk.meanr
xi = kk.xi


def gp_regression(x, new_x, y, kernel, y_err=None):
    
    if y_err is None:
        y_err =np.ones_like(y) *1e-10
    
    gp = skl_gp.GaussianProcessRegressor(kernel=kernel, alpha=y_err,
                                         optimizer=None,
                                         normalize_y=None)
    gp.fit(x,y)
    y_predict, y_std = gp.predict(new_x, return_std=True)
    return y_predict, y_std

@interact(sigma = widgets.FloatSlider(value=1.2, min=0.75, max=2.5, step=0.01, description='$\sigma$:',
          disabled=False,
          continuous_update=False,
          orientation='horizontal',
          readout=True,
          readout_format='.2f'), 
          l = widgets.FloatSlider(value=0.6, min=0.4, max=1.5, step=0.01, description='$l$:',
          disabled=False,
          continuous_update=False,
          orientation='horizontal',
          readout=True,
          readout_format='.2f'),
          kernel = widgets.Dropdown(options=['RBF', 'Matern'],
                                  value='RBF',
                                  description='Kernel:',
                                  disabled=False,))
def plot_samples(sigma, l, kernel):

    y_reduce = y[Filter]
    x_reduce = x[Filter]
    y_err_reduce = y_err[Filter]
    
    new_x = np.linspace(-55, 55, 500).reshape((500,1))
    Kernel = "%f * %s(%f)"%((sigma**2, kernel, l))
    Kernel = eval_kernel(Kernel)
    y_pred, y_std = gp_regression(x_reduce, new_x, y_reduce, 
                                  Kernel, y_err=y_err_reduce)

    gs = gridspec.GridSpec(1, 2, width_ratios=[1.5, 1])
    plt.figure(figsize=(20,8))
    plt.subplot(gs[0])
    
    # Data
    plt.scatter(x, y, c='b', label = 'data')
    plt.errorbar(x, y, linestyle='', yerr=y_err, ecolor='b', 
                 alpha=0.7,marker='.',zorder=0)
    
    # GP prediction
    plt.plot(new_x, y_pred, 'r', lw =3, label = 'GP prediction')
    plt.fill_between(new_x.T[0], y_pred-y_std, y_pred+y_std, color='r', alpha=0.3)
    
    plt.plot(new_x, np.zeros_like(new_x),'k--')
    plt.xlim(-55, 55)
    plt.ylim(-3.,3.)
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    plt.xlabel('X', fontsize=20)
    plt.ylabel('Y', fontsize=20)
    plt.legend(fontsize=18)
    
    plt.subplot(gs[1])
    
    distance = np.linspace(0, 2, 40)
    coord = np.array([distance, np.zeros_like(distance)]).T
    pcf = Kernel.__call__(coord, Y=np.zeros_like(coord))[:,0]
    
    plt.plot(distance, pcf, 'k', lw=3, label="Used correlation function")
    plt.scatter(delta_distance, xi, c='b', s=80, label="Measured 2-point correlation function")
    
    plt.ylim(0, 2.)
    plt.xlim(0, 2)
    plt.ylabel('$\\xi(|x_i-x_j|)$', fontsize=20)
    plt.xlabel('$|x_i-x_j|$', fontsize=20)
    plt.legend(fontsize=14)
    plt.title('Used correlation function (%s)'%(kernel), fontsize=16)

interactive(children=(FloatSlider(value=1.2, continuous_update=False, description='$\\sigma$:', max=2.5, min=0…